
https://en.wikipedia.org/wiki/Image_scaling#Algorithms

to do :
- own validation set
- resize images transformer (Bilinear done?)
- make noise transformer
-

In [2]:
#I don't think its working yet.
from __future__ import division
import math
from fractions import Fraction

from fuel.transformers import ExpectsAxisLabels, SourcewiseTransformer

class BilinearRescale(SourcewiseTransformer, ExpectsAxisLabels):
    """Resize an image to a fixed window size. Use bilinear interpolation with 4-relative nearest neighbors.
    Parameters
    ----------
    data_stream : :class:`AbstractDataStream`
        The data stream to wrap.
    window_shape : tuple
        The `(height, width)` tuple representing the size of the output
        window.
    Notes
    -----
    This transformer expects to act on stream sources which provide one of
     * Single images represented as 3-dimensional ndarrays, with layout
       `(channel, height, width)`.
     * Batches of images represented as lists of 3-dimensional ndarrays,
       possibly of different shapes (i.e. images of differing
       heights/widths).
     * Batches of images represented as 4-dimensional ndarrays, with
       layout `(batch, channel, height, width)`.
    The format of the stream will be un-altered, i.e. if lists are
    yielded by `data_stream` then lists will be yielded by this
    transformer.
    """
    def __init__(self, data_stream, image_shape, **kwargs):
        self.image_shape = image_shape
        kwargs.setdefault('produces_examples', data_stream.produces_examples)
        kwargs.setdefault('axis_labels', data_stream.axis_labels)
        super(BilinearRescale, self).__init__(data_stream, **kwargs)

    def transform_source_batch(self, source, source_name):
        self.verify_axis_labels(('batch', 'channel', 'height', 'width'),
                                self.data_stream.axis_labels[source_name],
                                source_name)
        height, width = self.image_shape
        print(source.shape)
        if isinstance(source, np.ndarray) and source.ndim == 4:
            # Not yet supported(batch, channels, height, width).
            raise Exception
        
        elif all(isinstance(b, np.ndarray) and b.ndim == 3 for b in source):
            return [self.transform_source_example(im, source_name)
                    for im in source]
        else:
            raise ValueError("uninterpretable batch format; expected a list "
                             "of arrays with ndim = 3, or an array with "
                             "ndim = 4")

    def transform_source_example(self, example, source_name):
        self.verify_axis_labels(('channel', 'height', 'width'),
                                self.data_stream.axis_labels[source_name],
                                source_name)
        height, width = self.image_shape
        nb_channel = example.shape[0] #That line could be replace by something more elegant.
        
        if not isinstance(example, np.ndarray) or example.ndim != 3:
            raise ValueError("uninterpretable example format; expected "
                             "ndarray with ndim = 3")
        image_height, image_width = example.shape[1:]
        #rescale_height, rescale_width = (image_height-1)/(height-1), (image_width-1)/(width-1)
        rescale_height, rescale_width = Fraction(image_height-1,height-1), Fraction(image_width-1,width-1)
        
        rescaled_image = np.zeros((nb_channel,height,width))
        #That code ain't pretty. Might do a cleaner version eventually
        
        for i,j in product(range(height), range(width)):
            x, y  = i*rescale_height, j * rescale_width
            x1 = np.array([math.floor(x), math.ceil(x)],dtype=np.intp)
            y1 = np.array([math.floor(y), math.ceil(y)],dtype=np.intp)
            dx,dy = x - x1[0], y - y1[0]
            del_x, del_y = x1[1]-x1[0], y1[1]-y1[0]
            
            #Stupid patching for float approximation induced problem
            if x1[1] == image_height:
                x1[1] -= 1
            if y1[1] == image_width:
                y1[1] -= 1
                
            if not x1[0] == x1[1] and not y1[0] == y1[1] :
                for c in range(nb_channel):
                    xy1,xy2 = example[c][np.ix_(x1,y1)]
                    x1y1,x2y1,x1y2,x2y2 = xy1[0],xy1[1],xy2[0],xy2[1]
                    rescaled_image[c,i,j] = (x2y1-x1y1)*dx/del_x + (x1y2 -x1y1) *dy/del_y + (x1y1 + x2y2 - x2y1 - x1y2) *dx/del_x*dy/del_y + x1y1
            elif x1[0] == x1[1] and y1[0] == y1[1]:
                rescaled_image[:,i,j] = example[:,x1[0],y1[0]]
            else:
                if y1[0] == y1[1]:
                    for c in range(nb_channel):
                        x1y,x2y = example[c][np.ix_(x1,y1)]
                        x1y,x2y = x1y[0], x1y[1]
                        rescaled_image[c,i,j] = (x2y-x1y)*dx/del_x + x1y
                else:
                    for c in range(nb_channel):
                        xy1,xy2 = example[c][np.ix_(x1,y1)]
                        xy1,xy2 = xy1[0], xy1[1]
                        rescaled_image[c,i,j] = (xy2 - xy1) * dy/del_y + xy1
            
        return rescaled_image
    

In [ ]:
#This section deals with the dogs_vs_cats problem with a simple MLP. It's mainly for testing the BilinearRescale

s0,s1 = 100,100
nb_hidden = 600
num_epochs = 1
batch_size = 1
num_batches = 25

# Let's load and process the dataset
import numpy as np
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers import Flatten

train = DogsVsCats(('train',), subset=slice(0, 20000))
test = DogsVsCats(('test',), subset=slice(0,100))

train_stream = DataStream.default_stream(
    train,
    iteration_scheme=ShuffledScheme(train.num_examples, batch_size)
)

test_stream = DataStream.default_stream(
    test,
    iteration_scheme=ShuffledScheme(test.num_examples, batch_size)
)

train_stream = BilinearRescale(train_stream, (s0,s1), which_sources=('image_features',))
test_stream = BilinearRescale(test_stream, (s0,s1), which_sources=('image_features',))

if True:
    train_stream = Flatten(train_stream, which_sources=('image_features',))
    test_stream = Flatten(test_stream, which_sources=('image_features',))

In [ ]:
import theano
from itertools import product
from theano import tensor
    
X = tensor.matrix('image_features')
T = tensor.lmatrix('targets')
W = theano.shared(
    np.random.uniform(low=-0.01, high=0.01, size=(s0*s1*3, nb_hidden)), 'W')
b = theano.shared(np.zeros(nb_hidden))
V = theano.shared(
    np.random.uniform(low=-0.01, high=0.01, size=(nb_hidden, 2)), 'V')
c = theano.shared(np.zeros(2))
params = [W, b, V, c]

H = tensor.nnet.sigmoid(tensor.dot(X, W) + b)
Y = tensor.nnet.softmax(tensor.dot(H, V) + c)

# Use Blocks to train this network
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import Printing
from blocks.extensions import FinishAfter, Timing, Printing, ProgressBar
from blocks.extensions.monitoring import (DataStreamMonitoring,
                                          TrainingDataMonitoring)
from blocks.bricks.cost import CategoricalCrossEntropy,MisclassificationRate
from blocks.monitoring import aggregation
from blocks.main_loop import MainLoop
from blocks.model import Model 

loss = tensor.nnet.categorical_crossentropy(Y, T.flatten()).mean()

#classification_error= MisclassificationRate().apply(tensor.argmax(Y),T)
classification_error= MisclassificationRate().apply(T.flatten(),Y)
#squared_error = SquaredError().apply(Y.T,tensor.eye(10)[T])

algorithm = GradientDescent(cost=loss, parameters=params,
                            step_rule=Scale(learning_rate=0.1))

# We want to monitor the cost as we train
loss.name = 'loss'
extensions = [Timing(), FinishAfter(after_n_epochs=num_epochs,
                              after_n_batches=num_batches),
              TrainingDataMonitoring([loss,classification_error],prefix="train",after_epoch=True),    
            #  DataStreamMonitoring(
           #           [loss],#classification_error
          #            test_stream,prefix="test"),
                  #ProgressBar(),
                 Printing(every_n_batches=1)]

#extensions = [Timing(),
#              FinishAfter(after_n_epochs=num_epochs),
#              TrainingDataMonitoring([loss,classification_error],prefix="train",after_epoch=True),
             #   DataStreamMonitoring([loss, classification_error],test_stream,prefix="test"),
             #    ProgressBar(),
#                  Printing() ]

#TrainingDataMonitoring([loss, classification_error,squared_error,aggregation.mean(algorithm.total_gradient_norm)], 
      #                  every_n_batches=5,
       #               prefix="train",
        #              after_epoch=True)
#model = Model(loss)

main_loop = MainLoop(data_stream=train_stream, algorithm=algorithm,
                     extensions=extensions)
main_loop.run()


In [ ]:
%debug

I was going try coding a CNN, but then i figured that i could probably reuse the LeNet example in blocks-example

Link to the depot : https://github.com/mila-udem/blocks-examples/blob/master/mnist_lenet/__init__.py

In [3]:
import logging
import numpy
from argparse import ArgumentParser

from theano import tensor

from blocks.algorithms import GradientDescent, Scale
from blocks.bricks import (Rectifier, Initializable,
                           Softmax)
from blocks.bricks.cost import CategoricalCrossEntropy, MisclassificationRate
from blocks.extensions import FinishAfter, Timing, Printing, ProgressBar
from blocks.extensions.monitoring import (DataStreamMonitoring,
                                          TrainingDataMonitoring)
from blocks.extensions.saveload import Checkpoint
from blocks.graph import ComputationGraph
from blocks.initialization import Constant, Uniform
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.monitoring import aggregation
from fuel.schemes import ShuffledScheme
from fuel.streams import DataStream
from toolz.itertoolz import interleave
from itertools import product
    
from LeNet import LeNet

def main_convnet(save_to, num_epochs, train, test, input_size=None, feature_maps=None, mlp_hiddens=None,
         conv_sizes=None, pool_sizes=None, batch_size=50, num_batches=None):
    if feature_maps is None:
        feature_maps = [20, 50]
    if mlp_hiddens is None:
        mlp_hiddens = [500]
    if conv_sizes is None:
        conv_sizes = [5, 5]
    if pool_sizes is None:
        pool_sizes = [2, 2]
    if input_size is None :
        input_size = (150, 150)
    output_size = 2
    
    # Use ReLUs everywhere and softmax for the final prediction
    conv_activations = [Rectifier() for _ in feature_maps]
    mlp_activations = [Rectifier() for _ in mlp_hiddens] + [Softmax()]
    convnet = LeNet(conv_activations, 3, input_size,
                    filter_sizes=zip(conv_sizes, conv_sizes),
                    feature_maps=feature_maps,
                    pooling_sizes=zip(pool_sizes, pool_sizes),
                    top_mlp_activations=mlp_activations,
                    top_mlp_dims=mlp_hiddens + [output_size],
                    border_mode='full',
                    weights_init=Uniform(width=.2),
                    biases_init=Constant(0))
    
    
    # We push initialization config to set different initialization schemes
    # for convolutional layers.
    convnet.push_initialization_config()
    convnet.layers[0].weights_init = Uniform(width=.2)
    convnet.layers[1].weights_init = Uniform(width=.09)
    convnet.top_mlp.linear_transformations[0].weights_init = Uniform(width=.08)
    convnet.top_mlp.linear_transformations[1].weights_init = Uniform(width=.11)
    convnet.initialize()
    logging.info("Input dim: {} {} {}".format(
        *convnet.children[0].get_dim('input_')))
    for i, layer in enumerate(convnet.layers):
        logging.info("Layer {} ({}) dim: {} {} {}".format(
            i, layer.__class__.__name__, *layer.get_dim('output')))

    x = tensor.tensor4('image_features')
    y = tensor.lmatrix('targets')

    # Normalize input and apply the convnet
    probs = convnet.apply(x)
    cost = CategoricalCrossEntropy().apply(y.flatten(),
            probs).copy(name='cost')
    error_rate = MisclassificationRate().apply(y.flatten(), probs).copy(
            name='error_rate')

    cg = ComputationGraph([cost, error_rate]) #2 cost?

    
    #Generating stream
    train_stream = DataStream.default_stream(
        train,
        iteration_scheme=ShuffledScheme(train.num_examples, batch_size)
    )

    test_stream = DataStream.default_stream(
        test,
        iteration_scheme=ShuffledScheme(test.num_examples, batch_size)
    )
    
    train_stream = BilinearRescale(train_stream, input_size, which_sources=('image_features',))
    test_stream = BilinearRescale(test_stream, input_size, which_sources=('image_features',))

    # Train with simple SGD
    algorithm = GradientDescent(
        cost=cost, parameters=cg.parameters,
        step_rule=Scale(learning_rate=0.1))
    # `Timing` extension reports time for reading data, aggregating a batch
    # and monitoring;
    # `ProgressBar` displays a nice progress bar during training.
    extensions = [Timing(),
                  FinishAfter(after_n_epochs=num_epochs,
                              after_n_batches=num_batches),
                  TrainingDataMonitoring(
                      [cost, error_rate,
                       aggregation.mean(algorithm.total_gradient_norm)],
                      prefix="train",
                      every_n_batches=1),
                  Checkpoint(save_to),
                  ProgressBar(),
                  Printing(every_n_batches=1)]

    model = Model(cost)

    main_loop = MainLoop(
        algorithm,
        train_stream,
        model=model,
        extensions=extensions)

    main_loop.run()


In [4]:
# Let's load and process the dataset
import numpy as np
from fuel.datasets.dogs_vs_cats import DogsVsCats

from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers import Flatten

# Load the training set
train = DogsVsCats(('train',), subset=slice(0, 200))
test = DogsVsCats(('test',), subset=slice(0,2000))


In [7]:
#main_convnet("lenetsave.txt",, batch_size=8, num_batches=5)

main_convnet("lenetsave.txt", 1, train, test, input_size=(150,150), feature_maps=None, mlp_hiddens=None,
         conv_sizes=None, pool_sizes=None, batch_size=1,num_batches=10)

NameError: global name 'BilinearRescale' is not defined

In [ ]:
%debug #Fucking hell ce bug. Jy comprends rien.

In [ ]:

import theano
from itertools import product
from blocks.graph import ComputationGraph
# Use Blocks to train this network
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import Printing
from blocks.extensions import FinishAfter, Timing, Printing, ProgressBar
from blocks.extensions.monitoring import (DataStreamMonitoring,
                                          TrainingDataMonitoring)
from blocks.bricks.cost import CategoricalCrossEntropy,MisclassificationRate
from blocks.monitoring import aggregation
from blocks.main_loop import MainLoop
from blocks.model import Model 

def main(save_to, model, num_epochs, train, test, input_size = (150,150), batch_size=50, learning_rate=0.05,flatten_stream=False,
         num_batches=10):

    #Variables theano
    x = tensor.matrix('image_features')
    T = tensor.lmatrix(name='targets')
    Y = mlp.apply(x)

    model.push_initialization_config()
    #model.children[0].weights_init = Constant(0.01)
    model.initialize()

    #Cost
#    loss = tensor.nnet.categorical_crossentropy(Y, T.flatten()).mean()
    #classification_error= MisclassificationRate().apply(tensor.argmax(Y),T)
    classification_error= MisclassificationRate().apply(T.flatten(),Y)
    #squared_error = SquaredError().apply(Y.T,tensor.eye(10)[T])
    cost = CategoricalCrossEntropy().apply(T.flatten(),Y)
    cg = ComputationGraph(cost)
    algorithm = GradientDescent(cost=cost, parameters=cg.parameters,
                             step_rule=Scale(learning_rate=learning_rate))
    



    #Generating stream
    train_stream = DataStream.default_stream(
        train,
        iteration_scheme=ShuffledScheme(train.num_examples, batch_size)
    )

    test_stream = DataStream.default_stream(
        test,
        iteration_scheme=ShuffledScheme(test.num_examples, batch_size)
    )


    #Rescale images
    train_stream = BilinearRescale(train_stream, input_size, which_sources=('image_features',))
    test_stream = BilinearRescale(test_stream, input_size, which_sources=('image_features',)) 
    
    #Flattening the stream
    if flatten_stream is True:
        train_stream = Flatten(train_stream, which_sources=('image_features',))
        test_stream = Flatten(test_stream, which_sources=('image_features',))

        
    #Monitoring
    cost.name = 'cost'
    extensions = [Timing(), FinishAfter(after_n_epochs=num_epochs,
                                  after_n_batches=num_batches),
                        TrainingDataMonitoring([cost,classification_error],prefix="train"),    

                      DataStreamMonitoring([cost, classification_error], test_stream,prefix="test"),
                      #ProgressBar(),
                     Printing(every_n_batches=1)]

    #TrainingDataMonitoring([loss, classification_error,squared_error,aggregation.mean(algorithm.total_gradient_norm)], 
          #                  every_n_batches=5,
           #               prefix="train",
            #              after_epoch=True)
    #model = Model(loss)

    main_loop = MainLoop(data_stream=train_stream, algorithm=algorithm,
                         extensions=extensions)
    main_loop.run()


#Model
from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor

input_size = (150,150)

mlp = MLP(activations=[Rectifier(name='rect0'),Logistic(name='sigmoid_1'),Softmax(name='softmax_2')], dims=[input_size[0]*input_size[1]*3, 1000, 500, 2],
          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))

main('mlp1.txt',mlp,2,train,test,input_size = input_size, batch_size=1, flatten_stream=True, num_batches=5)

- cuda lib_dirs error (it isnt in lib_dirs) cuda shared libra
LD_LIBRAIRY_
- Wierd import error with LeNet.
- retrieve information from test